In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
def convert_inputs_to_attention_format_fixed(target_text,params):
    #train_inputs = np.asarray(inputs)
    #train_inputs = process(train_inputs)
    #train_seq_len = train_inputs.shape[0]
    original = target_text.lower()
    target_text = '<start>'+ target_text.lower() + '<end>'
    target_text = target_text.lower().split(' ')
    # Transform char into index
    targets = np.asarray([params.dictionary[x] for x in target_text])
    return  targets, original

def get_dictionary_fixed(label_dir, params):
    with open(label_dir, 'r') as txt:
        data = txt.readlines()
    data = data[:params.number_sentence]
    word_list = []
    raw_text = []
    for line in data:
        sentence = re.sub(r"[^a-zA-Z]+", ' ', line)
        words_vector = []
        sentence = sentence.split(' ')
        for word in sentence:
            if word != '':
                words_vector.append(word.lower())
                word_list.append(word.lower())
        raw_text.append(words_vector)
    word_list = pd.Series(word_list).unique()
    dict = {'<start>': 1, '<end>': 2}
    for word in word_list:
        if word not in dict:
            dict[word] = len(dict) + 1
    return dict

def pad_sequences(sequences, maxlen=None, dtype=np.float32,
                  padding='post', truncating='post', value=0.):
    '''Pads each sequence to the same length: the length of the longest
    sequence.
        If maxlen is provided, any sequence longer than maxlen is truncated to
        maxlen. Truncation happens off either the beginning or the end
        (default) of the sequence. Supports post-padding (default) and
        pre-padding.
        Args:
            sequences: list of lists where each element is a sequence
            maxlen: int, maximum length
            dtype: type to cast the resulting sequence.
            padding: 'pre' or 'post', pad either before or after each sequence.
            truncating: 'pre' or 'post', remove values from sequences larger
            than maxlen either in the beginning or in the end of the sequence
            value: float, value to pad the sequences to the desired value.
        Returns
            x: numpy array with dimensions (number_of_sequences, maxlen)
            lengths: numpy array with the original sequence lengths
    '''
    lengths = np.asarray([len(s) for s in sequences], dtype=np.int64)

    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x, lengths


In [4]:
from model import *
from utils import *
import tensorflow as tf
import numpy as np
import pandas as pd
import time
from jiwer import wer as jwer
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
class parameters():

    number_sentence = 8854
    num_layers = 4
    d_model = 256
    dff = 1024
    num_heads = 16
    input_vocab_size = 0
    target_vocab_size = 0
    dropout_rate = 0.1
    freq = 100
    feature = 1
    data = 'eeg'
    seed = 1234
    n_batches = 10
    epochs = 50
    prenet = True

"""
Get dataset and parameters
"""
tf.random.set_seed(params.seed)
params.dictionary = get_dictionary_fixed('sentences.txt',params)
label_dir = 'sentences.txt'
target_set = []
with open(label_dir, 'r') as txt:
    data = txt.readlines()
    for line in data:
        sentence = re.sub(r"[^a-zA-Z]+", ' ', line)
        target_set.append(convert_inputs_to_attention_format_fixed(sentence,params)[0])
input_set = np.random.rand(8854, 700, 13)
input_set, inp_seq_len = pad_sequences(input_set, dtype=np.float32)
target_set, target_seq_len = pad_sequences(target_set, dtype=np.int64)
params.target_vocab_size = len(params.dictionary)+1
params.max_length = len(target_set[0])

ModuleNotFoundError: No module named 'unidecode'